In [1]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import time
import imageio
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [2]:
data_path = os.path.join(Path(os.getcwd()).parent.absolute(), "Data")
# video_path = os.path.join(data_path, "Suitcase.mp4")
video_path = os.path.join(data_path, "cut.mp4")
# video_path = os.path.join(data_path, "cut3.mp4")
vidcap = cv2.VideoCapture(video_path)
success, image = vidcap.read()

# get two frames of the video
count = 0
while success and count < 53:
    if count > 50:
        cv2.imwrite("frame%d.jpg" % count, image) 
        print('Read a new frame', count)
    success, image = vidcap.read()
    count += 1

Read a new frame 51
Read a new frame 52


In [3]:
# Video cut
# ffmpeg_extract_subclip(video_path, 10, 15, targetname=os.path.join(data_path,"cut3.mp4"))

In [4]:
# #sift
# sift = cv2.SIFT_create()

# #feature matching
# bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# img1 = cv2.imread('frame51.jpg')
# img2 = cv2.imread('frame52.jpg')

# img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
# keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)

# matches = bf.match(descriptors_1, descriptors_2)
# matches = sorted(matches, key = lambda x:x.distance)

# sift = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, matches[:300], img2, flags=2)

# cv2.imshow('SIFT', sift)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
#sift
sift = cv2.SIFT_create()

#feature matching
bf = cv2.BFMatcher.create(cv2.NORM_L2, crossCheck=True)

cap = cv2.VideoCapture(video_path)
cap2 = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(os.path.join(data_path, "sift.mp4"), fourcc, 10.0, (1280,360))

# length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

img_lst = []
frame_number = 0
frame_shift = 1

while success:
    current_frame = cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    next_frame = cap2.set(cv2.CAP_PROP_POS_FRAMES, frame_number+frame_shift)
    suc, current_frame = cap.read()
    suc, next_frame = cap2.read()

    if next_frame is None :
        break

    start = time.time()

    current_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    next_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)        

    keypoints_1, descriptors_1 = sift.detectAndCompute(current_frame,None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(next_frame,None)

    matches = bf.match(descriptors_1,descriptors_2)
    matches = sorted(matches, key = lambda x:x.distance)

    end = time.time()
    totalTime = end - start

    fps = 1 / totalTime
    #print("FPS: ", fps)
    
    img3 = cv2.drawMatches(current_frame, keypoints_1, next_frame, keypoints_2, matches[200:250], next_frame, flags=2)
    img_lst.append(img3)

    # cv2.putText(img3, f'FPS: {int(fps)}', (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
    cv2.imshow('SIFT', img3)

    frame_number += 1

    out.write(img3)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
out.release()
cap.release()

# imageio.mimsave(os.path.join(data_path, "sift.gif"), img_lst)